# 1. Camera Calibration

This script intends to calibrate a camera using an object.\
For this implementation we will use the openCV method for calibration(https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)

**Import the libraries**

In [4]:
from platform import python_version

print(python_version())

3.8.5


In [5]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
import cv2

#for LM method
from scipy.optimize import curve_fit
#matplotlib widget

In [6]:
if False:
    folder_dir = "C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/right_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    print(list_images)

In [7]:
if False:
    folder_dir = "C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_2_meters_tivera/left_2/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    print(list_images)

### Run through image directory

In [22]:

if True:
        
    import os 
    import string
    from os import listdir

    camera_name = 'right'
    # get the path/directory
    folder_dir = f"C:/Users/UX550/OneDrive - Universidade de Aveiro/Documentos/5º ANO\Thesis/calibration_photos/calibration_4_tivera_1/{camera_name}_4/"
    img_cnt = 0
    # list of images
    list_images = os.listdir(folder_dir)
    new_list = sorted(list_images,key = lambda k: int(k.split('_')[2].split('.')[0]))

    for images in new_list:
        # check if the image ends with png
        renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
        print(images)
        print(renamed_image)
        # If equal, no renaming
        if(images == renamed_image):
            print("equal")
        else:
            while True:
                print(images)
                renamed_image = camera_name + "_init_" + '%s' % img_cnt + '.png'
                print(renamed_image)
                try:
                    os.rename(folder_dir + '/' + images,folder_dir + '/' + renamed_image)
                except FileExistsError:
                    img_cnt += 1
                    pass
                else:
                    break
                
            
        img_cnt += 1


right_init_0.png
right_init_0.png
equal
right_init_1.png
right_init_1.png
equal
right_init_2.png
right_init_2.png
equal
right_init_3.png
right_init_3.png
equal
right_init_4.png
right_init_4.png
equal
right_init_5.png
right_init_5.png
equal
right_init_6.png
right_init_6.png
equal
right_init_7.png
right_init_7.png
equal
right_init_8.png
right_init_8.png
equal
right_init_9.png
right_init_9.png
equal
right_init_10.png
right_init_10.png
equal
right_init_11.png
right_init_11.png
equal
right_init_12.png
right_init_12.png
equal
right_init_13.png
right_init_13.png
equal
right_init_14.png
right_init_14.png
equal
right_init_15.png
right_init_15.png
equal
right_init_16.png
right_init_16.png
equal
right_init_17.png
right_init_17.png
equal
right_init_18.png
right_init_18.png
equal
right_init_19.png
right_init_19.png
equal
right_init_20.png
right_init_20.png
equal
right_init_21.png
right_init_21.png
equal
right_init_22.png
right_init_22.png
equal
right_init_23.png
right_init_23.png
equal
right_init_2

## Zhang's Method
Comparing to the DLT method, the Zhang method only calculate the intrinsic parameters. We instead only need to find 5 parameters.

## 2. Calibration

## Find the homographies

In [23]:
def cornerfinding(n_imgs,patternsize,dist, img_name):
    '''
    @param
    [in] n_imgs : number of images used
    [in] patternsize : size of the checkerboard
    [in] dist : distance between squares
    [in] img_name : name of the jpg
    [out] imagecoo : imagecoordinates
    [out] worldcoo : worldcoordinates
    '''
    # number of images
    n_imgs = n_imgs
    # pattern size
    patternsize= patternsize
    #distance between squares(in mm)
    dist = dist

    # specified number of iterations are completed.
    criteria = (cv2.TERM_CRITERIA_EPS +
                cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Vector for 3D points
    threedpoints = []

    # Vector for 2D points
    twodpoints = []

    #3D points real world coordinates( static -> chessboard)
    # create a matrix of zeros for 48 points , each with 3 coordinates
    world_coo = np.zeros((1, patternsize[0]* patternsize[1],3), np.float32)
    world_coo[:, :, :2] = np.mgrid[0:patternsize[0], 0:patternsize[1]].T.reshape(-1, 2)
    # Transform coordinates in mm
    world_coo = world_coo*dist

    #prev_img_shape = None

    for i in range(n_imgs):

        im = i

        #Import the image
        name = (str(img_name)+ str(im)+".png")
        img = cv2.imread(name)
        #print(name)
        # convert it to gray scale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        retval, corners= cv2.findChessboardCorners(gray,patternsize,None)
        #print(corners)
        if(retval == True):
            threedpoints.append(world_coo)
            #print(name)

            # Refining pixel coordinates or given 2d points.
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria).reshape(patternsize[0]*patternsize[1],2)

            twodpoints.append(corners2)
            # Draw and display the corners
            image = cv2.drawChessboardCorners(img,patternsize,corners2,retval)
            #plt.imshow(image)
            #plt.show()
        else:
            print(name)

    # image coordinates and world coordinates for the diferent images( the world coordinates are always the same)
    imagecoo = np.asarray(twodpoints)
    worldcoo = np.asarray(threedpoints) 
    return imagecoo, worldcoo

#C:\Users\UX550\OneDrive - Universidade de Aveiro\Documentos\5º ANO\Thesis\calibration\images
#name_left = ("../calibration_photos/calibration_photos_2meters/left_160/left_init_")
#name_right =("../calibration_photos/calibration_photos_2meters/right_160/right_init_")
#name_left = ("../calibration_photos/calibration_photos/left_edited/left_init_")
#name_right =("../calibration_photos/calibration_photos/right_edited/right_init_")
name_left = "../Thesis/calibration_photos/calibration_4_tivera_1/left_4/left_init_"
name_right = ("../Thesis/calibration_photos/calibration_4_tivera_1/right_4/right_init_")

#name_right = ("../calibration_photos/calibration_2_meters_tivera/right_2/right_init_")
#name_left = ("calibration_photos/left_edited/left_init_")
#name_right = ("calibration_photos/right_edited/right_init_")
n_imgs = 180
square_size = 44
pattern_size = (12,8)
imagecoo_left,worldcoo_left = cornerfinding(n_imgs,pattern_size,square_size,name_left)
imagecoo_right,worldcoo_right = cornerfinding(n_imgs,pattern_size,square_size,name_right)


In [24]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L,image_size):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)

    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC
    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,distL,distR,rvecs_L,rvecs_R

In [25]:
worldcoo_left.shape

(180, 1, 96, 3)

In [26]:
type(imagecoo_right[0,0,0])

numpy.float32

In [27]:
#image_size = (1936,1216)
#ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,dist_L,dist_R,rvecs_L,rvecs_R = calib(worldcoo_left,imagecoo_right,imagecoo_left,image_size)
#retS


In [ ]:
image_size = (1936,1216)


ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []
P_R_org_list = []
P_L_org_list = []
dist_L_list = []
dist_R_list = []
rvecs_L_list = []
rvecs_R_list = []

ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat,dist_L,dist_R,rvecs_L,rvecs_R = calib(worldcoo_left,imagecoo_right,imagecoo_left,image_size)


ret_R_list.append([ret_R])
mtx_R_list.append([mtx_R])
ret_L_list.append([ret_L])
mtx_L_list.append([mtx_L])
retS_list.append([retS])
new_mtxL_list.append([new_mtxL])
new_mtxR_list.append([new_mtxR])
Rot_list.append([Rot])
Trns_list.append([Trns])
Emat_list.append([Emat])
Fmat_list.append([Fmat])
dist_L_list.append(dist_L)
dist_R_list.append(dist_R)
rvecs_L_list.append(rvecs_L)
rvecs_R_list.append(rvecs_R)

In [16]:
P_L_list = []
P_R_list = []
R1_list = []
R2_list = []
R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL, ret_L, new_mtxR, ret_R, (image_size[0],image_size[1]), Rot, Trns,flags = cv2.CALIB_ZERO_DISPARITY)

R1_list.append(R1)
R2_list.append(R2)
P_L_list.append(P_L_est)
P_R_list.append(P_R_est)

In [17]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [18]:
R2

array([[0.965324, 0.024442, -0.259907],
       [-0.024538, 0.999695, 0.002879],
       [0.259898, 0.003599, 0.965629]])

In [19]:
%store R1,R2,retS_list,P_L_est,new_mtxL_list,ret_L_list,dist_L_list,P_R_est,new_mtxR_list,ret_R_list,dist_R_list,Rot_list,Trns_list >".\calibration_files\4_meters\calib_4_{n_imgs}.txt"

Writing 'R1,R2,retS_list,P_L_est,new_mtxL_list,ret_L_list,dist_L_list,P_R_est,new_mtxR_list,ret_R_list,dist_R_list,Rot_list,Trns_list' (tuple) to file '.\calibration_files\4_meters\calib_4_125.txt'.


In [110]:
new_mtxL

array([[2377.283484, 0.000000, 899.953008],
       [0.000000, 2367.931216, 640.622319],
       [0.000000, 0.000000, 1.000000]])

In [108]:
new_mtxR

array([[2.55806726e+03, 0.00000000e+00, 9.81954052e+02],
       [0.00000000e+00, 2.56132169e+03, 6.37192983e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [109]:
P_R_est

array([[ 2.55982889e+03,  0.00000000e+00,  9.30319321e+02,
        -1.21479659e+06],
       [ 0.00000000e+00,  2.55982889e+03,  6.22341766e+02,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00]])

In [110]:
P_L_est

array([[2.55982889e+03, 0.00000000e+00, 9.30319321e+02, 0.00000000e+00],
       [0.00000000e+00, 2.55982889e+03, 6.22341766e+02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [113]:
Trns

array([[-474.06210272],
       [   1.62292259],
       [ -21.70802313]])

In [71]:
Rot

array([[ 9.97888679e-01,  5.96478491e-03, -6.46730722e-02],
       [-5.95512156e-03,  9.99982210e-01,  3.42188894e-04],
       [ 6.46739627e-02,  4.36695836e-05,  9.97906447e-01]])

In [70]:
np.savetxt(f"../calibration_photos/calibration_2_meters_tivera/calib_2_5.txt",R1,R2,P_L_est,P_R_est,retS,new_mtxL,new_mtxR,ret_L,ret_R,dist_L,dist_R,Rot,Trns)

TypeError: _savetxt_dispatcher() takes from 2 to 9 positional arguments but 14 were given

In [88]:
P_L_est

array([[5.64744350e+03, 0.00000000e+00, 8.04407046e+04, 0.00000000e+00],
       [0.00000000e+00, 5.64744350e+03, 7.13646637e+02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [90]:
mtx_R

array([[8.63279725e+03, 0.00000000e+00, 9.72224667e+02],
       [0.00000000e+00, 8.64721221e+03, 6.10718092e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
mtx_R

array([[3.11402075e+03, 0.00000000e+00, 9.84769693e+02],
       [0.00000000e+00, 3.11732577e+03, 6.61750985e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

0.3304236193635975